In [1]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.auxiliary_functions as aux
import src.condition as con
import src.base as base
from itertools import combinations
from src.spread import get_spread
import src.amplitude as amplitude
import src.change as change

## Veri Hazırlama

In [2]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [3]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [4]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [5]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [7]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [8]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">

In [10]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [11]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [12]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])

## Change

In [13]:
#change alınıyor
df_change=base.groupby_date_time(df_spread).apply(change.get_change)
df_change=df_change.droplevel([0,1])

## Amplitude

In [31]:
amplitude_list=[]
for pair in df_change.columns:
    signed=change.sign(df_change[pair])
    duration=signed.groupby('sign').apply(change.last_time)
    df=signed.groupby('sign').transform('sum')
    amp=df.loc[duration]
    amplitude_list.append(amp)

c:\users\ugur.eren\python codes\environments\inflation_env\lib\site-packages\pandas\core\series.py:856: RuntimeWarning: invalid value encountered in sign
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [32]:
df_amplitude = pd.concat(amplitude_list,axis=1)

# Sonuçlar Excele Yazdırılıyor

In [46]:
df_mid_price.to_excel('out/mid_price.xlsx')
df_spread.to_excel('out/spread.xlsx')
df_change.to_excel('out/change.xlsx')
df_amplitude.to_excel('out/amplitude.xlsx')